In [ ]:
from hipars import Sorting
import random
import numpy as np

total_side_length = (80,250)
filled_shape = np.array([80,90],np.int32)
state_array = np.ndarray(total_side_length,dtype=bool)

for i in range(total_side_length[0]):
    for j in range(total_side_length[1]):
        state_array[i,j] = random.random() < 0.2

row_start = (total_side_length[0] - filled_shape[0]) // 2
col_start = (total_side_length[1] - filled_shape[1]) // 2
comp_zone = (row_start, row_start + filled_shape[0], col_start, col_start + filled_shape[1])

target_geometry = np.full(filled_shape,False,dtype=bool)
for i in range(filled_shape[0]):
    if i % 4 == 2 or i % 4 == 3:
        for j in range(filled_shape[1] // 15):
            for j2 in range(15*j+2,15*j+13,2):
                target_geometry[i,j2] = True

state_array_copy = state_array.copy()

sorter = Sorting.Sorting()
sorter.configure_log(log_file_name="test.log")

# Parallel sorting
parallel_moves = sorter.sort_parallel_lattice_by_row(state_array_copy, (comp_zone[0], comp_zone[1]), (comp_zone[2], comp_zone[3]), target_geometry)
sorter.flush_logs()
if parallel_moves is not None:
    for move in parallel_moves:
        print("Move shifts " + str(len(move.steps[0].rowSelection)) + " rows and " + \
            str(len(move.steps[0].colSelection)) + " columns for " + str(len(move.steps) - 1) + " steps")

Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 16 columns for 1 steps
Move shifts 3 rows and 2 columns for 1 steps
Move shifts 1 rows and 16 columns for 2 steps
Move shifts 1 rows and 16 columns for 2 steps
Move shifts 1 rows and 16 columns for 2 steps
Move shifts 1 rows and 4 columns for 2 steps
Move shifts 1 rows and 16 columns for 2 steps
Move shifts 1 rows and 3 columns for 2 steps
Move shifts 1 rows and 16 columns for 3 steps
Move shifts 1 rows and 14 columns for 3 steps
Move shifts 1 rows and 16 columns for 2 steps
Move shifts 1 rows and 16 columns for 2 steps
Move shifts 1 rows and 1 columns for 2 steps
Move shifts 1 rows and 16 columns for 3 steps
Move shifts 1 rows and 9 columns for 3

In [ ]:
# Export video of movement sequence, takes quite long to execute

import numpy as np
import cv2
from ipywidgets import IntProgress
from IPython.display import display

spacing = (10,5)
radius = 2
size = np.array(state_array.shape) * np.array(spacing)
channels = 3

row_arange = np.arange(size[0])
col_arange = np.arange(size[1])

def image_coords(row,col):
    image_row = int(spacing[0] / 2 + spacing[0] * row)
    image_col = int(spacing[1] / 2 + spacing[1] * col)
    inside = (row_arange[:,None] - image_row) ** 2 + (col_arange[None, :] - image_col) ** 2 <= (radius ** 2)
    return inside

duration = 2
fps = 30

pause_between_moves = 2
pause_between_submoves = 1
frames_per_spacing = 0.1
total_frames = 2 * fps
for move in parallel_moves:
    last_step = move.steps[0]
    for step in move.steps[1:]:
        max_col_dist = np.max(np.abs(np.array(step.colSelection) - np.array(last_step.colSelection))) * spacing[1]
        max_row_dist = np.max(np.abs(np.array(step.rowSelection) - np.array(last_step.rowSelection))) * spacing[0]
        max_dist = np.max([max_col_dist,max_row_dist])
        total_frames += max_dist * frames_per_spacing + pause_between_submoves
        last_step = step
    total_frames += pause_between_moves - pause_between_submoves
total_frames = int(np.ceil(total_frames))

f = IntProgress(min=0, max=total_frames)
display(f)

out = cv2.VideoWriter('latticeMovement.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]))
data = np.full((*size,3), 255, dtype='uint8')
for row in range(total_side_length[0]):
    for col in range(total_side_length[1]):
        if state_array[row,col]:
            data[image_coords(row,col),...] = 0
frame_in_move = 0
active_move = None
move_index = 0

state_array_movement_copy = state_array.copy()

for frame in range(total_frames):
    # If no active move: Pause and then select next move
    if active_move is None and frame_in_move >= pause_between_moves and move_index < len(parallel_moves):
        active_move = parallel_moves[move_index]
        atom_movement_matrix = np.ndarray((len(active_move.steps[0].rowSelection),len(active_move.steps[0].colSelection)), bool)
        for ri,row in enumerate(active_move.steps[0].rowSelection):
            for ci,col in enumerate(active_move.steps[0].colSelection):
                if state_array_movement_copy[int(row + 0.25),int(col + 0.25)]:
                    atom_movement_matrix[ri,ci] = True
                    data[image_coords(row,col),...] = 255
                    state_array_movement_copy[int(row + 0.25),int(col + 0.25)] = False
                else:
                    atom_movement_matrix[ri,ci] = False
        frame_in_move = 0
    move_data = data.copy()
    if active_move is not None:
        remaining_frames = frame_in_move
        last_step = active_move.steps[0]
        still_in_move = False
        for i,step in enumerate(active_move.steps[1:]):
            max_row_dist = np.max(np.abs(np.array(step.rowSelection) - np.array(last_step.rowSelection))) * spacing[0]
            max_col_dist = np.max(np.abs(np.array(step.colSelection) - np.array(last_step.colSelection))) * spacing[1]
            max_dist = np.max([max_col_dist,max_row_dist])
            frames_in_submove = max_dist * frames_per_spacing
            if remaining_frames < frames_in_submove:
                move_fraction = remaining_frames / frames_in_submove
                rows = np.array(last_step.rowSelection) + (np.array(step.rowSelection) - np.array(last_step.rowSelection)) * move_fraction
                cols = np.array(last_step.colSelection) + (np.array(step.colSelection) - np.array(last_step.colSelection)) * move_fraction
                for ri,row in enumerate(rows):
                    for ci,col in enumerate(cols):
                        if atom_movement_matrix[ri,ci]:
                            move_data[image_coords(row,col),0:2] = 0
                break
            else:
                remaining_frames -= frames_in_submove
                if remaining_frames < pause_between_submoves:
                    if i < len(active_move.steps) - 2:
                        for ri,row in enumerate(step.rowSelection):
                            for ci,col in enumerate(step.colSelection):
                                if atom_movement_matrix[ri,ci]:
                                    move_data[image_coords(row,col),0:2] = 0
                        break
                    else:
                        frame_in_move = 0
                        for ri,row in enumerate(active_move.steps[-1].rowSelection):
                            for ci,col in enumerate(active_move.steps[-1].colSelection):
                                if atom_movement_matrix[ri,ci] and row >= 0 and row < state_array.shape[0] and col >= 0 and col < state_array.shape[1]:
                                    data[image_coords(row,col),...] = 0
                                    state_array_movement_copy[int(row + 0.25),int(col + 0.25)] = True
                        move_data = data.copy()
                        active_move = None
                        move_index += 1
                else:
                    remaining_frames -= pause_between_submoves
            last_step = step
    frame_in_move += 1
    out.write(move_data)
    f.value += 1
out.release()

IntProgress(value=0, max=20009)